In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import random
import math

Загружаем датасет

In [25]:
df = pd.read_csv('src/boston.csv')

Преобразуем данные в массив numpy

In [30]:
X_np = df.drop(df.columns[-1], axis=1).to_numpy()
y_np = df['MEDV'].to_numpy()

Функция для разбиения на тренировочные и тестовые данных для массивов numpy

In [28]:
def train_test_split_np(
        X: np.ndarray, 
        y: np.ndarray,
        test_size: float = 0.2
        ) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Разделяет массивы признаков X и целевой переменной y на обучающую и тестовую выборки.

     :param X: np.ndarray
        Массив признаков формы (n_samples, n_features).
    ":param y: np.ndarray
        Массив целевых значений формы (n_samples,).
    :param test_size: float, по умолчанию 0.2
        Доля данных, выделяемая на тестовую выборку. Должна быть в интервале (0, 1).
    :return X_train: np.ndarray, обучающая выборка признаков.
    :return X_test: np.ndarray, тестовая выборка признаков.
    :return y_train: np.ndarray, обучающая выборка целевых значений.
    :return y_test: np.ndarray, тестовая выборка целевых значений.
    :raise ValueError: Возникает, если количество строк в X
        и y не совпадает или test_size не в (0, 1).
        """
    if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
        raise TypeError('X и y должны быть массивами numpy')
    # Проверка входных данных
    if X.shape[0] != y.shape[0]:
        raise ValueError("Количество строк в X и y должно совпадать.")
    if not 0 < test_size < 1:
        raise ValueError("test_size должен быть между 0 и 1.")

    n_samples = X.shape[0] 
    n_test_samples = int(n_samples * test_size)
    
    # Перестановка
    indices = np.random.permutation(n_samples)
    
    # Разделяем индексы
    test_indices = indices[:n_test_samples]
    train_indices = indices[n_test_samples:]
    
    # Разделяем данные
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test

Функции линейной регрессии и предсказания

In [32]:
def linear_regression_np(
        X: np.ndarray,
        y: np.ndarray
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Обучает линейную регрессию с использованием градиентного спуска.

    :param X: Матрица признаков формы (m, n), где m — количество примеров,
        n — количество признаков.
    :param y: Вектор целевых значений формы (m,).
    :return: Кортеж из трёх элементов:
        - weights: Вектор весов модели (включая свободный член) формы (n + 1,).
        - X_mean: Вектор средних значений по каждому признаку,
            использованный для нормализации.
        - X_std: Вектор стандартных отклонений по каждому признаку,
            использованный для нормализации.
    """
    # Подготовка данных: нормализация признаков
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    # Избегаем деления на ноль
    X_std[X_std == 0] = 1
    X_normalized = (X - X_mean) / X_std
    
    # Добавление столбца единиц для свободного члена (intercept)
    m, n = X_normalized.shape
    X_b = np.column_stack([np.ones(m), X_normalized])

    learning_rate = 0.001
    weights = np.random.randn(n + 1) * 0.01

    for i in range(50000):
        y_pred = X_b @ weights
        error = y_pred - y

        gradient = (2 / m) * X_b.T @ error
        weights = weights - learning_rate * gradient

    return weights, X_mean, X_std

def predict_np(X, weights, X_mean, X_std):
    """
    Выполняет предсказание на новых данных с использованием обученной модели линейной регрессии.
    
    :param X: Матрица признаков для предсказания формы (m, n).
    :param weights: Вектор весов модели (включая свободный член) формы (n + 1,).
    :param X_mean: Вектор средних значений признаков, полученный при обучении.
    :param X_std: Вектор стандартных отклонений признаков, полученный при обучении.
    :return: Вектор предсказанных значений формы (m,).
    """
    X_normalized = (X - X_mean) / X_std
    m = X_normalized.shape[0]
    X_b = np.column_stack([np.ones(m), X_normalized])
    return X_b @ weights


Разделяем данные на тренировочные и тестовые

In [31]:
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split_np(X_np, y_np)

Обучение модели

In [ ]:
weights_np, X_mean_np, X_std_np = linear_regression_np(X_train_np, y_train_np)

Получение предсказаний

In [ ]:
y_predicts_np = predict_np(X_test_np, weights_np, X_mean_np, X_std_np)

Получение предиктов с помощью sklearn

In [37]:
model = LinearRegression()
model.fit(X_train_np, y_train_np)
y_pred_sk = model.predict(X_test_np)

Сравнение предиктов и действительных y

In [42]:
print('my\t\t\t\tsklearn\t\t\t\tTrue')
for i in range(len(y_predicts_np)):
    print(f'{y_predicts_np[i]}\t\t{y_pred_sk[i]}\t\t{y_test_np[i]}')


my				sklearn				True
19.91916975345094		19.919994229753733		20.3
17.963041971611716		17.963385395698147		20.0
25.91301500678168		25.91354229952288		23.3
8.98395746579368		8.983366930880003		5.0
28.638593592950755		28.63967686117953		25.0
20.58451687746171		20.583459047082165		21.7
28.097692595856973		28.0970034229693		31.2
19.866059599680753		19.865457238928137		23.0
43.220004665266714		43.22018696718774		50.0
7.894211891238246		7.894336360509197		7.2
15.652328493684733		15.652071432081016		15.6
21.431257488803944		21.4311653933646		21.7
15.08769765453872		15.087556354596757		8.4
28.48180011490275		28.48201484997628		26.6
13.929281153827972		13.928678057366188		14.3
19.813608868648853		19.81374090472209		27.5
21.848594127707642		21.847925766644497		17.0
8.81772437610297		8.817437135622534		14.6
40.938116519387165		40.93912222779575		48.5
21.571638484568396		21.571985529395022		18.9
15.91182672626632		15.911991638235502		14.1
17.67223289667278		17.672392323806147		15.4
32.1039522025568

Преобразуем данные в обычные массивы

In [46]:
X_lst = X_np.tolist()
y_lst = y_np.tolist()

Функция разделения данных на тренировочные и тестовые для обычных массивов

In [43]:
def train_test_split_lst(
        X: list,
        y: list,
        test_size: float = 0.2
        ) -> tuple[list, list, list, list]:
    """
    Разделяет данные на обучающую и тестовую выборки.
    
    :param X: Список признаков или входных данных
    :param y: Список целевых значений или меток
    :param test_size: Доля данных для тестовой выборки (от 0 до 1), по умолчанию 0.2
    :return: Кортеж из четырех списков (X_train, X_test, y_train, y_test)
    :raises ValueError: Если количество элементов в X и y не совпадает
                       или если test_size не находится в диапазоне (0, 1)
    """

    # Проверка входных данных
    if len(X) != len(y):
        raise ValueError('Количество элементов X и y должно совпадать')
    if not 0 < test_size < 1:
        raise ValueError('test_size должен быть между 0 и 1')

    n_samples = len(X)
    n_test_samples = int(n_samples * test_size)

    # Случайная перестановка индексов
    indexes = list(range(n_samples))
    random.shuffle(indexes)

    test_indexes = indexes[:n_test_samples]
    train_indexes = indexes[n_test_samples:]

    # Разделение данных
    X_train = [X[i] for i in train_indexes]
    X_test = [X[i] for i in test_indexes]
    y_train = [y[i] for i in train_indexes]
    y_test = [y[i] for i in test_indexes]

    return X_train, X_test, y_train, y_test    

Функции линейной регрессии и предсказаний на основе данных из обычных массивов

In [151]:
def linear_regression_lst(
        X: list,
        y: list,
        learning_rate: float = 0.0001,
        n_iterations: int = 50000
        ) -> tuple[list, list, list]:
    """
    Линейная регрессия методом градиентного спуска.
    
    :param X: Список списков признаков (двумерный массив)
    :param y: Список целевых значений (одномерный массив)
    :param learning_rate: Скорость обучения для градиентного спуска, по умолчанию 0.001
    :param n_iterations: Количество итераций градиентного спуска, по умолчанию 50000.
    :return: Кортеж из трех элементов (weights, X_mean, X_std)
    """
    m = len(X)
    n = len(X[0])

    # Нормализация признаков
    # Среднее значение по каждому признаку
    X_mean = []
    for j in range(n):
        total = sum(X[i][j] for i in range(m))
        X_mean.append(total / m)

    # Стандартное отклонение по каждому признаку
    X_std = []
    for j in range(n):
        variance = sum((X[i][j] - X_mean[j]) ** 2 for i in range(m)) / m
        std = math.sqrt(variance)
        if std == 0:
            std = 1.0
        X_std.append(std)

    # Нормализация признаков
    X_normalized = []
    for i in range(m):
        row = []
        for j in range(n):
            normalized_val = (X[i][j] - X_mean[j]) / X_std[j]
            row.append(normalized_val)
        X_normalized.append(row)

    # Добавление столбца единиц для свободного члена (intercept)
    X_b = []
    for i in range(m):
        row_with_bias = [1.0] + X_normalized[i] 
        X_b.append(row_with_bias)
    
    # Инициализация весов
    weights = []
    for _ in range(n + 1):
        # Генерирация случайных весов 
        w = (random.random() - 0.5) * 0.1
        weights.append(w)

    # Градиентный спуск
    for _ in range(n_iterations):
        y_pred = []
        for i in range(m):
            pred = 0.0
            for j in range(n + 1):
                pred += X_b[i][j] * weights[j]
            y_pred.append(pred)

        errors = [y_pred[i] - y[i] for i in range(m)]

        gradient = []
        for i in range(n + 1):
            grad_sum = 0.0
            for j in range(m):
                grad_sum += X_b[j][i] * errors[j]
            gradient.append((2.0 / m) * grad_sum)

        for i in range(n + 1):
            weights[i] = weights[i] - learning_rate * gradient[i]


    return weights, X_mean, X_std

def predict_lst(
        X: list, weights: list, X_mean: list, X_std: list
    ) -> list:
    """
    Делает предсказания на основе обученной модели линейной регрессии.
    
    :param X: Список списков признаков для предсказания
    :param weights: Веса модели, полученные из linear_regression_lst
    :param X_mean: Средние значения признаков, использованные при обучении
    :param X_std: Стандартные отклонения признаков, использованные при обучении
    :return: Список предсказанных значений
    """
    m = len(X)
    n = len(X[0])

    # Нормализция признаков
    X_normalized = []
    for i in range(m):
        row = []
        for j in range(n):
            normalized_val = (X[i][j] - X_mean[j]) / X_std[j]
            row.append(normalized_val)
        X_normalized.append(row)
    
    # Добавление столбца единиц для свободного члена
    X_b = []
    for i in range(m):
        X_b.append([1.0] + X_normalized[i])
    
    # Вычисление предсказаний
    y_pred = []
    for i in range(m):
        pred = 0.0
        for j in range(n + 1):
            pred += X_b[i][j] * weights[j]
        y_pred.append(pred)
    
    return y_pred

Разделение данных на тестовые и тренировочные

In [142]:
X_train_lst, X_test_lst, y_train_lst, y_test_lst = train_test_split_lst(X_lst, y_lst)

Обучение модели

In [152]:
weights_lst, X_mean_lst, X_std_lst =linear_regression_lst(X_train_lst, y_train_lst)

Получение предиктов

In [153]:
y_predicts_lst = predict_lst(X_test_lst, weights_lst, X_mean_lst, X_std_lst)

Сравнение предиктов

In [154]:
print('my\t\t\t\tsklearn\t\t\t\tTrue')
for i in range(len(y_predicts_np)):
    print(f'{y_predicts_lst[i]}\t\t{y_pred_sk[i]}\t\t{y_test_np[i]}')

my				sklearn				True
32.24725845925987		19.919994229753733		20.3
21.079653735432803		17.963385395698147		20.0
26.542122669492734		25.91354229952288		23.3
26.478058565674928		8.983366930880003		5.0
32.91130422258298		28.63967686117953		25.0
12.82572042903925		20.583459047082165		21.7
22.108549150892756		28.0970034229693		31.2
39.658791163068976		19.865457238928137		23.0
16.845110290242573		43.22018696718774		50.0
23.408057443527404		7.894336360509197		7.2
25.70134616185357		15.652071432081016		15.6
17.918733057040974		21.4311653933646		21.7
14.812034190035115		15.087556354596757		8.4
19.34657540088738		28.48201484997628		26.6
28.27104635205341		13.928678057366188		14.3
14.767169510745294		19.81374090472209		27.5
36.16185512102464		21.847925766644497		17.0
41.00184246935716		8.817437135622534		14.6
16.43275708043991		40.93912222779575		48.5
30.26563725140527		21.571985529395022		18.9
26.475639255124033		15.911991638235502		14.1
40.78606106541205		17.672392323806147		15.4
16.004150784142

Из полученных результатов видно, что модель плохо обучилась с помощью этой функции. 
Почему?

Реализация на первый взгляд корректна. 
При изменении шага и количества итераций результат особо не изменяется 

Проверим функцию на более простых данных

In [114]:
from sklearn.datasets import make_regression

X, y, coeffs = make_regression(n_samples=50, n_features=1, n_informative=1,
                                                noise=10, coef=True, random_state=11)


In [141]:
Xx = X.tolist()
yy = y.tolist()

X_tr, X_t, y_tr, y_t = train_test_split_lst(Xx, yy)

ww, xm, xs = linear_regression_lst(X_tr, y_tr)

yw = predict_lst(X_t, ww, xm, xs)

In [139]:
print('my\t\t\t\tTrue')
for i in range(len(yw)):
    print(f'{yw[i]}\t\t{y_t[i]}')

my				True
6.565734377438252		17.48650736914131
-38.8683291621623		-33.956083075144896
59.669097627970686		49.05026172271842
-30.398879497165353		-34.991616801854605
60.19984950172021		40.524686315692456
-99.74857882931593		-91.73785524267645
89.78250075868205		91.76970817317603
24.709279027354647		21.196446429226846
5.7661153529072635		-12.779819901095298
-38.149305321563695		-22.641901500695617


Данные все равно сильно расходятся

Что не так?